In [10]:
import numpy as np

def kodla(data, H):
  """
  LDPC kodlayıcı fonksiyonu.

  Args:
      data: Kodlanacak 16 bitlik veri.
      H: Parite kontrol matrisi.

  Returns:
      Kodlanmış veri.
  """

  k = len(data)
  n = H.shape[1]
  x = np.concatenate((data, np.zeros(n - k)))

  # Parite kontrol denklemlerini kullanarak kodlama
  for i in range(H.shape[0]):
    # H matrisindeki her satır için
    satir = H[i]
    # Matrisin ilgili sütunlarındaki verilerin toplamını hesaplama
    toplam = np.dot(satir, x)
    # Eğer toplam 1 ise, ilgili parite bitini değiştirme
    if toplam % 2 == 1:
      # **Düzeltme:** 
      # Parite bitinin indeksini doğru şekilde belirle
      parite_bit_index = np.where(satir == 1)[0][0]  
      # ilgili parite bitini değiştir
      x[parite_bit_index] = 1 - x[parite_bit_index]

  return x

def geri_al(alınan_veri, H, max_iterasyon=50):
  """
  Belief Propagation (İnanç Yayılımı) algoritması kullanarak LDPC kod çözücü.

  Args:
      alınan_veri: Alınan kodlanmış veri.
      H: Parite kontrol matrisi.
      max_iterasyon: Maksimum iterasyon sayısı.

  Returns:
      Çözülmüş veri.
  """

  n = H.shape[1]
  x = np.copy(alınan_veri)

  # İteratif olarak mesajları ileterek kod çözme
  for _ in range(max_iterasyon):
    # Mesajları düğümler arasında iletmek için döngü
    for i in range(n):
      # Her bir veri düğümü için
      for j in range(H.shape[0]):
        # H matrisinde 1 olan satır ve sütun indekslerini bulma
        if H[j, i] == 1:
          # Düğümden gelen mesajı hesaplama
          mesaj = hesapla_mesaj(H, x, j, i)
          # Mesajı ilgili parite kontrol düğümüne gönderme
          x[i] = mesaj

    # Tüm mesajlar iletildikten sonra, en olası değerlere göre karar verme
    x = np.round(x)

  # Kodlanmış veriden orijinal veriyi çıkarma
  return x[:16]

def hesapla_mesaj(H, x, j, i):
  """
  Bir veri düğümünden bir parite kontrol düğümüne mesaj hesaplar.

  Args:
      H: Parite kontrol matrisi.
      x: Veri vektörü.
      j: Parite kontrol düğümü indeksi.
      i: Veri düğümü indeksi.

  Returns:
      Hesaplanan mesaj.
  """

  # Parite kontrol düğümüne bağlı diğer veri düğümlerinin indekslerini bulma
  diğer_düğümler = np.where(H[j] == 1)[0]
  diğer_düğümler = diğer_düğümler[diğer_düğümler != i]

  # Diğer veri düğümlerinden gelen mesajların toplamını hesaplama
  toplam = 0
  for k in diğer_düğümler:
    toplam += x[k]

  # Mesajı hesaplama
  mesaj = 1 - (toplam % 2)

  return mesaj

# 16 bitlik örnek veri
data = np.array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1])

H = np.array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
              [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],  # Yeni satır
              [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],  # Yeni satır
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]])  # Yeni satır

# Veriyi kodlama
kodlanmış_veri = kodla(data, H)

# Gürültü ekleme (örnek olarak 2 bit değiştirilmiş)
gürültülü_veri = np.copy(kodlanmış_veri)
gürültülü_veri[2] = 1 - gürültülü_veri[2]
gürültülü_veri[5] = 1 - gürültülü_veri[5]

# Veriyi çözme
çözülmüş_veri = geri_al(gürültülü_veri, H, max_iterasyon=100) # İterasyon sayısını artırdık

print("Orijinal veri:", data)
print("Kodlanmış veri:", kodlanmış_veri)
print("Gürültülü veri:", gürültülü_veri)
print("Çözülmüş veri:", çözülmüş_veri)

Orijinal veri: [1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1]
Kodlanmış veri: [1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
Gürültülü veri: [1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
Çözülmüş veri: [0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]


In [11]:
import numpy as np

# Parite kontrol matrisi
H = np.array([[1, 0, 1, 0, 1, 0],
              [0, 1, 1, 1, 0, 0],
              [1, 1, 0, 0, 0, 1]])

# Mesaj
mesaj = np.array([1, 0, 1])

# Kodlama
kodlanmış_kelime = np.dot(mesaj, H.transpose())

# Gürültü ekleme
gürültülü_kelime = kodlanmış_kelime.copy()
gürültülü_kelime[2] = 0  # 3. bitte hata ekleme

# Çözme (İteratif Mesaj Geçişi ile)
def belief_propagation(gürültülü_kelime, H):
  """İteratif Mesaj Geçişi algoritmasını kullanarak LDPC çözme."""
  n = len(gürültülü_kelime)
  m = H.shape[0]
  
  # Mesajları başlat
  mesajlar_düğümlerden_kontrol_nodlarına = np.zeros((n, m))
  mesajlar_kontrol_nodlarından_düğümlere = np.zeros((n, m))

  # İterasyonlar
  for _ in range(10):  # İterasyon sayısını ayarlayabilirsiniz
    # Düğümden kontrol noduna mesajlar
    for i in range(n):
      for j in range(m):
        if H[j, i] == 1:
          mesajlar_düğümlerden_kontrol_nodlarına[i, j] = gürültülü_kelime[i]
          for k in range(m):
            if k != j and H[k, i] == 1:
              mesajlar_düğümlerden_kontrol_nodlarına[i, j] ^= mesajlar_kontrol_nodlarından_düğümlere[i, k]

    # Kontrol nodundan düğüme mesajlar
    for j in range(m):
      for i in range(n):
        if H[j, i] == 1:
          mesajlar_kontrol_nodlarından_düğümlere[i, j] = 0
          for k in range(n):
            if k != i and H[j, k] == 1:
              mesajlar_kontrol_nodlarından_düğümlere[i, j] ^= mesajlar_düğümlerden_kontrol_nodlarına[k, j]

  # Çözüm
  çözüm = np.zeros(n)
  for i in range(n):
    for j in range(m):
      if H[j, i] == 1:
        çözüm[i] ^= mesajlar_kontrol_nodlarından_düğümlere[i, j]
  
  return çözüm

çözüm = belief_propagation(gürültülü_kelime, H)

# Sonuçlar
print("Orijinal mesaj:", mesaj)
print("Kodlanmış kelime:", kodlanmış_kelime)
print("Gürültülü kelime:", gürültülü_kelime)
print("Çözülen mesaj:", çözüm)

ValueError: shapes (3,) and (6,3) not aligned: 3 (dim 0) != 6 (dim 0)

In [24]:
>>> import numpy as np
>>> from pyldpc import make_ldpc, encode, decode, get_message
>>> n = 16
>>> d_v = 4
>>> d_c = 8
>>> snr = 20
>>> H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
>>> k = G.shape[1]
>>> v = np.random.randint(2, size=k)
>>> y = encode(G, v, snr)
>>> d = decode(H, y, snr)
>>> x = get_message(G, d)
>>> assert abs(x - v).sum() == 0

In [25]:
k

11

In [26]:
v

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1])

In [27]:
y

array([ 1.06425412,  0.94640448,  0.92090987, -0.91762292,  0.80672584,
        1.06328019,  1.06844534,  1.139491  ,  1.05198552, -1.12621343,
       -0.96011364, -0.92503552,  1.04277694,  1.05605957,  1.05192846,
        0.95173482])

In [28]:
d

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0])

In [29]:
x

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1])

In [19]:
k

6